In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import datasets
from src import config

In [ ]:
hyper_parameters = config.HyperParameters()

dataset: datasets.Dataset = datasets.load_dataset(
    hyper_parameters.data_config.dataset, split="train"
)  # type: ignore

In [ ]:
dataset[:3]

In [ ]:
type(dataset[:3])

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
from PIL import Image
from typing import Any


def collate_fn(batch: list[dict[str, Image.Image | str]]) -> dict[str, list[list[dict[str, Any]]]]:
    images = [
        [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "image": data_instance["image"],
                    },
                ],
            }
        ]
        for data_instance in batch
    ]
    texts = [
        [
            {
                "role": "user",
                "content": [{"type": "text", "text": data_instance["caption"]}],
            }
        ]
        for data_instance in batch
    ]
    return {
        "images": images,
        "texts": texts,
    }


dl = DataLoader(
    dataset=dataset,
    batch_size=16,
    shuffle=True,
    pin_memory=True,
    drop_last=True,
    collate_fn=collate_fn,
)

In [ ]:
batch = next(iter(dl))

In [ ]:
from torch.utils.data import Dataset

isinstance(dataset, datasets.Dataset)

In [ ]:
type(dataset)

In [ ]:
batch

In [ ]:
from src import data, config, model

import torch

In [ ]:
hyper_parameters = config.HyperParameters()
train_dl, valid_dl = data.get_dataset(hyper_parameters)

In [ ]:
batch = next(iter(train_dl))

In [ ]:
len(batch["images"])

In [ ]:
model_components = model.init_model(hyper_parameters.llm_model_config, device=torch.device("mps"))

In [ ]:
import torch.nn as nn

isinstance(model_components.model, nn.Module)

In [ ]:
batch.keys()

In [ ]:
with torch.no_grad():
    text_embeddings = model.embed_text(batch["texts"], model_components)
    # image_embeddings = model.embed_image(batch["images"], model_components)

In [ ]:
(
    text_embeddings.shape,
    # image_embeddings.shape,
    (text_embeddings**2).sum(),
    # (image_embeddings**2).sum(),
)

In [ ]:
lora_model = model.get_lora_model(model_components.model, hyper_parameters)

In [ ]:
model_components.model = lora_model

with torch.no_grad():
    text_embeddings = model.embed_text(batch["texts"], model_components)


In [ ]:
(
    text_embeddings.shape,
    # image_embeddings.shape,
    (text_embeddings**2).sum(),
    # (image_embeddings**2).sum(),
)

In [9]:
1

1